# Import

In [2]:
import json
import trimesh
import numpy as np
import os
import open3d as o3d

import plotly.graph_objs as go

from scipy.spatial import ConvexHull

import pdb

from tqdm import tqdm


def load_pcd_vis(pcd_path,pcd_name=None,sub_sample=50000) -> np.ndarray:
    _, file_extension = os.path.splitext(pcd_path)
    if file_extension == ".pcd":
        pcd = o3d.io.read_point_cloud(pcd_path)
        point_clouds_data = np.asarray(pcd.points)
    elif file_extension == ".txt":
        point_clouds_data = np.loadtxt(pcd_path)
    elif file_extension == ".npy":
        point_clouds_data = np.load(pcd_path)
    else:
        raise NotImplementedError
    
    N = point_clouds_data.shape[0]
    point_clouds_data = point_clouds_data[np.random.choice(N, sub_sample, replace=True), :]
    
    x = point_clouds_data[:, 0]
    y = point_clouds_data[:, 1]
    z = point_clouds_data[:, 2]
    trace = go.Scatter3d(x=x, y=y, z=z, 
            mode='markers',     
            marker=dict(
                size=0.5,  # Adjust the size of the markers here
                color='rgba(35, 35, 250, 0.8)'  # Set the color you want (e.g., light blue)
            ),
            name=pcd_name)
    return trace

def quaternion_list2rotmat(quant_list: list, format="xyzw"):
    assert len(quant_list) == 4, "Quaternion needs 4 elements"
    if format=="xyzw":
        q = np.quaternion(quant_list[0], quant_list[1], quant_list[2], quant_list[3])
    elif format=="wxyz":
        q = np.quaternion(quant_list[1], quant_list[2], quant_list[3], quant_list[0])
    R = quaternion.as_rotation_matrix(q)
    return R

def mesh_apply_rts(mesh, rotation_mat_c2w=np.eye(3), translation_c2w=np.zeros(3), scale_c2w=np.ones(3), mesh_name=None, mesh_color=None):
    vertices = np.array(mesh.vertices)
    faces = np.array(mesh.faces)
    
    transformed_vertices = (rotation_mat_c2w @ vertices.T).T - translation_c2w[np.newaxis, :]
    transformed_vertices = vertices * scale_c2w
    
    x, y, z = transformed_vertices.T  # Transposed for easier unpacking
    i, j, k = faces.T  # Unpack faces

    if mesh_color is None:
        mesh_color = "rgba(244,22,100,0.5)"

    mesh_transformed = go.Mesh3d(
        x=x, y=y, z=z,
        i=i, j=j, k=k,
        opacity=0.5,
        color=mesh_color,
        name=mesh_name
    )
    return mesh_transformed

def plotly_mesh3d_to_trimesh(plotly_mesh3d):
    """
    将Plotly Mesh3d对象转换为trimesh.Trimesh对象。
    
    参数:
        plotly_mesh3d (plotly.graph_objs.Mesh3d): Plotly Mesh3d对象。
        
    返回:
        trimesh.Trimesh: 转换后的trimesh对象。
    """
    # 从Plotly Mesh3d对象提取顶点坐标
    vertices = np.column_stack((plotly_mesh3d.x, plotly_mesh3d.y, plotly_mesh3d.z))
    
    # 从Plotly Mesh3d对象提取面
    faces = np.column_stack((plotly_mesh3d.i, plotly_mesh3d.j, plotly_mesh3d.k))
    
    # 创建trimesh.Trimesh对象
    trimesh_obj = trimesh.Trimesh(vertices=vertices, faces=faces)
    
    return trimesh_obj

def BoundingCubeNormalization(mesh,buffer=1.03):
    """
    transferred from deepsdf preprocessing
    input: mesh: trimesh.load
    """
    vertices = np.array(mesh.vertices)
    faces = np.array(mesh.faces)

    xmax = vertices[:,0].max()
    ymax = vertices[:,1].max()
    zmax = vertices[:,2].max()

    xmin = vertices[:,0].min()
    ymin = vertices[:,1].min()
    zmin = vertices[:,2].min()

    xcenter = (xmax+xmin)/2.0
    ycenter = (ymax+ymin)/2.0
    zcenter = (zmax+zmin)/2.0

    vertices[:,0] -= xcenter
    vertices[:,1] -= ycenter 
    vertices[:,2] -= zcenter

    norms = np.linalg.norm(vertices, axis=1)

    max_dist = norms.max()
    max_dist *= buffer

    vertices /= max_dist

    x, y, z = vertices.T  # Transposed for easier unpacking
    i, j, k = faces.T  # Unpack faces

    # if mesh_color is None:
    mesh_color = "rgba(244,22,100,0.5)"
    mesh_name = "normalized"

    mesh_normalized = go.Mesh3d(
        x=x, y=y, z=z,
        i=i, j=j, k=k,
        opacity=0.5,
        color=mesh_color,
        name=mesh_name
    )
    
    return mesh_normalized, 1./max_dist, np.array([xcenter,ycenter,zcenter])

def mesh_load_go(mesh_path, scale_c2w=None, rotation_quat_wxyz=None, translation_c2w=None,mesh_name=None):
    # 从文件加载网格数据
    file_suffix = mesh_path.split(".")[-1]
    if file_suffix == "obj":
        with open(mesh_path, 'r') as file:
            lines = file.readlines()

        vertices = []
        faces = []

        for line in lines:
            if line.startswith('v '):
                vertex = line.split()[1:]
                vertices.append([float(vertex[0]), float(vertex[1]), float(vertex[2])])
            elif line.startswith('f '):
                face = line.split()[1:]
                face_indices = [int(idx.split('/')[0]) - 1 for idx in face]
                faces.append(face_indices)

        mesh = go.Mesh3d(x=[v[0] for v in vertices], y=[v[1] for v in vertices], z=[v[2] for v in vertices],
                        i=[f[0] for f in faces], j=[f[1] for f in faces], k=[f[2] for f in faces], name=mesh_name)
        return mesh

    elif file_suffix == "ply":
        from plyfile import PlyData

        # 从PLY文件加载网格数据
        plydata = PlyData.read(mesh_path)

        # 提取顶点坐标
        vertices = np.array([list(vertex) for vertex in plydata['vertex'].data])

        # 提取面数据
        faces = np.array(plydata['face'].data['vertex_indices'])
        faces = np.array([list(row) for row in faces])

        # 创建网格图形对象
        mesh = go.Mesh3d(x=vertices[:, 0], y=vertices[:, 1], z=vertices[:, 2], i=faces[:, 0], j=faces[:, 1], k=faces[:, 2], name=mesh_name)
        return mesh
def load_mesh(file_path):
    # 加载mesh文件
    return trimesh.load(file_path, force='mesh')

def mesh_to_points(mesh, count=100000):
    # 将mesh表面采样为点集
    return mesh.sample(count)

def compute_convex_hull_volume(points):
    # 计算点集的凸包体积
    hull = ConvexHull(points)
    return hull.volume

def compute_iou(mesh1, mesh2, sample_count=100000):
    # 将两个mesh表面采样为点集
    points1 = mesh_to_points(mesh1, count=sample_count)
    points2 = mesh_to_points(mesh2, count=sample_count)
    
    # 分别计算两个点集的凸包体积
    volume1 = compute_convex_hull_volume(points1)
    volume2 = compute_convex_hull_volume(points2)
    
    # 计算两个点集的并集体积
    union_points = np.concatenate([points1, points2], axis=0)
    union_volume = compute_convex_hull_volume(union_points)
    
    # 计算交集体积（使用容斥原理）
    intersection_volume = volume1 + volume2 - union_volume
    
    # 计算IoU
    iou = intersection_volume / union_volume
    return iou

# from evaluation_metrics import load_mesh, mesh_to_points, compute_convex_hull_volume, compute_iou

label2id = {
        "sofa": "04256520",
        "table": "04379243",
        "bed": "02818832",
        "bathtub": "02808440",
        "chair": "03001627", # 03001627
        "cabinet": "02933112",
        'plane':'02691156',
        'bottle':'02876657',
    }

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
label = "sofa"
gt_root = "/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus"
gt_root = os.path.join(gt_root,label2id[label])

sdf_root = "/storage/user/huju/transferred/ws_dditnach/DeepSDF/data/SdfSamples/canonical_mesh_manifoldplus" # 加载相关的transformation

output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/stage2_diff_cond_scanarcw/recon/49999/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond/recon/23999/Meshes"

mesh_list = os.listdir(output_root)
ious = []
base_names = []

for id_mesh, mesh in tqdm(enumerate(mesh_list)):
    if id_mesh < 6:
        continue

    mesh_path1 = os.path.join(output_root,mesh)

    base_name = os.path.basename(mesh_path1).split(".")[0]
    base_names.append(base_name)

    mesh_id, scene1, scene2, _,  obj_id = base_name.split(".")[0].split("_")
    
    # pdb.set_trace()
    # 加载两个mesh文件
    # mesh1 = load_mesh(mesh_path1)
    mesh1 = load_mesh(mesh_path1)

    # mesh_path2 = os.path.join(gt_root,base_name, "model_normalized_manifoldplus.obj")
    mesh_path2 = os.path.join(gt_root,base_name,"model_canonical_manifoldplus.obj")
    mesh2 = load_mesh(mesh_path2)
    
    sdf_path = os.path.join(sdf_root,label2id[label],base_name+".npz")
    sdf = np.load(sdf_path)
    translation_mesh2sdf = sdf['translation_mesh2sdf']
    scale_mesh2sdf = sdf["scale_mesh2sdf"]

    mesh1 = mesh_apply_rts(mesh1,mesh_name="recon",mesh_color="rgba(22,244,244,0.5)")
    # mesh2 = mesh_apply_rts(mesh2,translation_c2w=translation_mesh2sdf,scale_c2w=scale_mesh2sdf,mesh_name="gt")
    mesh2, scale, translation = BoundingCubeNormalization(mesh2)
    # mesh2 = mesh_apply_rts(mesh2,mesh_name="gt")

    # input()

    print(mesh_path1)
    print(mesh_path2)

    # vis_data = [mesh1,mesh2]

    # layout = go.Layout(scene=dict(
    #         aspectmode='data',  # Set the aspect ratio to 'cube' for equal scales
    #         xaxis_title='X',
    #         yaxis_title='Y',
    #         zaxis_title='Z',
    #     ),)

    # fig = go.Figure()
    # fig = go.Figure(data=vis_data, layout=layout)
    # fig.show()

    # 计算IoU
    mesh1 = plotly_mesh3d_to_trimesh(mesh1)
    mesh2 = plotly_mesh3d_to_trimesh(mesh2)
    iou = compute_iou(mesh1, mesh2)
    print(f"IoU: {iou}")
    ious.append(iou)

    # input()

    continue

0it [00:00, ?it/s]

/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond/recon/23999/Meshes/2ee577ae974272dc9faa444e7f5fd692_scene0244_01_ins_3.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/04256520/2ee577ae974272dc9faa444e7f5fd692_scene0244_01_ins_3/model_canonical_manifoldplus.obj


7it [00:00, 13.51it/s]

IoU: 0.7151529420656857
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond/recon/23999/Meshes/f98615e51a596fc5c05575120a46cd3b_scene0394_00_ins_8.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/04256520/f98615e51a596fc5c05575120a46cd3b_scene0394_00_ins_8/model_canonical_manifoldplus.obj
IoU: 0.7237128522579209
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond/recon/23999/Meshes/ef479941cb60405f8cbd400aa99bee96_scene0549_00_ins_8.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/04256520/ef479941cb60405f8cbd400aa99bee96_scene0549_00_ins_8/model_canonical_manifoldplus.obj


9it [00:02,  3.58it/s]

...

IoU: 0.9510125804191538
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond/recon/23999/Meshes/c856e6b37c9e12ab8a3de2846876a3c7_scene0334_02_ins_1.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/04256520/c856e6b37c9e12ab8a3de2846876a3c7_scene0334_02_ins_1/model_canonical_manifoldplus.obj


94it [01:46,  1.05s/it]

IoU: 0.39563961257210695
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond/recon/23999/Meshes/c304687f0778986c6f1e631ee7d027b9_scene0326_00_ins_1.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/04256520/c304687f0778986c6f1e631ee7d027b9_scene0326_00_ins_1/model_canonical_manifoldplus.obj


94it [01:48,  1.15s/it]


KeyboardInterrupt: 

# Sofa

## ddit_stage2_diff_cond_sofa_train_neighbor

In [ ]:
label = "sofa"
gt_root = "/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus"
gt_root = os.path.join(gt_root,label2id[label])

sdf_root = "/storage/user/huju/transferred/ws_dditnach/DeepSDF/data/SdfSamples/canonical_mesh_manifoldplus" # 加载相关的transformation

# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/stage2_diff_cond_scanarcw/recon/49999/Meshes"
# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond/recon/23999/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_neighbor/output/49999/test/mesh/Meshes"

mesh_list = os.listdir(output_root)
ious = []
base_names = []

for id_mesh, mesh in tqdm(enumerate(mesh_list)):
    if id_mesh < 6:
        continue

    mesh_path1 = os.path.join(output_root,mesh)

    base_name = os.path.basename(mesh_path1).split(".")[0]
    base_names.append(base_name)

    mesh_id, scene1, scene2, _,  obj_id = base_name.split(".")[0].split("_")
    
    # pdb.set_trace()
    # 加载两个mesh文件
    # mesh1 = load_mesh(mesh_path1)
    mesh1 = load_mesh(mesh_path1)

    # mesh_path2 = os.path.join(gt_root,base_name, "model_normalized_manifoldplus.obj")
    mesh_path2 = os.path.join(gt_root,base_name,"model_canonical_manifoldplus.obj")
    mesh2 = load_mesh(mesh_path2)
    
    sdf_path = os.path.join(sdf_root,label2id[label],base_name+".npz")
    sdf = np.load(sdf_path)
    translation_mesh2sdf = sdf['translation_mesh2sdf']
    scale_mesh2sdf = sdf["scale_mesh2sdf"]

    mesh1 = mesh_apply_rts(mesh1,mesh_name="recon",mesh_color="rgba(22,244,244,0.5)")
    # mesh2 = mesh_apply_rts(mesh2,translation_c2w=translation_mesh2sdf,scale_c2w=scale_mesh2sdf,mesh_name="gt")
    mesh2, scale, translation = BoundingCubeNormalization(mesh2)
    # mesh2 = mesh_apply_rts(mesh2,mesh_name="gt")

    # input()

    print(mesh_path1)
    print(mesh_path2)

    # vis_data = [mesh1,mesh2]

    # layout = go.Layout(scene=dict(
    #         aspectmode='data',  # Set the aspect ratio to 'cube' for equal scales
    #         xaxis_title='X',
    #         yaxis_title='Y',
    #         zaxis_title='Z',
    #     ),)

    # fig = go.Figure()
    # fig = go.Figure(data=vis_data, layout=layout)
    # fig.show()

    # 计算IoU
    mesh1 = plotly_mesh3d_to_trimesh(mesh1)
    mesh2 = plotly_mesh3d_to_trimesh(mesh2)
    iou = compute_iou(mesh1, mesh2)
    print(f"IoU: {iou}")
    ious.append(iou)

    # input()

    continue

: 

In [9]:
# ious = np.array(ious)
print(ious.mean())

0.46858645722563635


## ddit_stage2_diff_cond_sofa_train_noneighbor

In [17]:
label = "sofa"
gt_root = "/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus"
gt_root = os.path.join(gt_root,label2id[label])

sdf_root = "/storage/user/huju/transferred/ws_dditnach/DeepSDF/data/SdfSamples/canonical_mesh_manifoldplus" # 加载相关的transformation

# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/stage2_diff_cond_scanarcw/recon/49999/Meshes"
# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond/recon/23999/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_neighbor/output/49999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_noneighbor/output/69999/test/mesh/Meshes"

mesh_list = os.listdir(output_root)
ious = []
base_names = []

for id_mesh, mesh in tqdm(enumerate(mesh_list)):

    mesh_path1 = os.path.join(output_root,mesh)

    base_name = os.path.basename(mesh_path1).split(".")[0]
    base_names.append(base_name)

    mesh_id, scene1, scene2, _,  obj_id = base_name.split(".")[0].split("_")
    
    # pdb.set_trace()
    # 加载两个mesh文件
    # mesh1 = load_mesh(mesh_path1)
    mesh1 = load_mesh(mesh_path1)

    # mesh_path2 = os.path.join(gt_root,base_name, "model_normalized_manifoldplus.obj")
    mesh_path2 = os.path.join(gt_root,base_name,"model_canonical_manifoldplus.obj")
    mesh2 = load_mesh(mesh_path2)
    
    sdf_path = os.path.join(sdf_root,label2id[label],base_name+".npz")
    sdf = np.load(sdf_path)
    translation_mesh2sdf = sdf['translation_mesh2sdf']
    scale_mesh2sdf = sdf["scale_mesh2sdf"]

    mesh1 = mesh_apply_rts(mesh1,mesh_name="recon",mesh_color="rgba(22,244,244,0.5)")
    # mesh2 = mesh_apply_rts(mesh2,translation_c2w=translation_mesh2sdf,scale_c2w=scale_mesh2sdf,mesh_name="gt")
    mesh2, scale, translation = BoundingCubeNormalization(mesh2)
    # mesh2 = mesh_apply_rts(mesh2,mesh_name="gt")

    # input()

    print(mesh_path1)
    print(mesh_path2)

    # vis_data = [mesh1,mesh2]

    # layout = go.Layout(scene=dict(
    #         aspectmode='data',  # Set the aspect ratio to 'cube' for equal scales
    #         xaxis_title='X',
    #         yaxis_title='Y',
    #         zaxis_title='Z',
    #     ),)

    # fig = go.Figure()
    # fig = go.Figure(data=vis_data, layout=layout)
    # fig.show()

    # 计算IoU
    mesh1 = plotly_mesh3d_to_trimesh(mesh1)
    mesh2 = plotly_mesh3d_to_trimesh(mesh2)
    iou = compute_iou(mesh1, mesh2)
    print(f"IoU: {iou}")
    ious.append(iou)

    # input()

0it [00:00, ?it/s]

/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_noneighbor/output/69999/test/mesh/Meshes/f98615e51a596fc5c05575120a46cd3b_scene0394_00_ins_8.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/04256520/f98615e51a596fc5c05575120a46cd3b_scene0394_00_ins_8/model_canonical_manifoldplus.obj


1it [00:00,  1.03it/s]

IoU: 0.5336437548752274
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_noneighbor/output/69999/test/mesh/Meshes/7ae657b39aa2be68ccd1bcd57588acf8_scene0362_01_ins_3.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/04256520/7ae657b39aa2be68ccd1bcd57588acf8_scene0362_01_ins_3/model_canonical_manifoldplus.obj


2it [00:01,  1.03it/s]

...

IoU: 0.9055676042539331
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_noneighbor/output/69999/test/mesh/Meshes/fb74336a6192c4787afee304cce81d6f_scene0536_01_ins_3.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/04256520/fb74336a6192c4787afee304cce81d6f_scene0536_01_ins_3/model_canonical_manifoldplus.obj


38it [00:29,  1.47it/s]

IoU: 0.7875945795984995
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_noneighbor/output/69999/test/mesh/Meshes/58447a958c4af154942bb07caacf4df3_scene0474_03_ins_2.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/04256520/58447a958c4af154942bb07caacf4df3_scene0474_03_ins_2/model_canonical_manifoldplus.obj


39it [00:29,  1.30it/s]

IoU: 0.23549040706430352


In [19]:
del iou_path
del mesh_list_path

ious = np.array(ious)
print(ious.mean())
iou_path = output_root.replace("/Meshes","/ious.txt")
np.savetxt(iou_path,ious)

mesh_list_path = output_root.replace("/Meshes","/mesh_list.txt")
with open(mesh_list_path,"w") as f:
    for imesh in mesh_list:
        f.write(imesh+"\n")

0.6360272986451388


## ddit_stage2_sofa

### 13999

In [20]:
label = "sofa"
gt_root = "/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus"
gt_root = os.path.join(gt_root,label2id[label])

sdf_root = "/storage/user/huju/transferred/ws_dditnach/DeepSDF/data/SdfSamples/canonical_mesh_manifoldplus" # 加载相关的transformation

# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/stage2_diff_cond_scanarcw/recon/49999/Meshes"
# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond/recon/23999/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_neighbor/output/49999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_noneighbor/output/69999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/13999/test/mesh/Meshes"

mesh_list = os.listdir(output_root)
ious = []
base_names = []

for id_mesh, mesh in tqdm(enumerate(mesh_list)):

    mesh_path1 = os.path.join(output_root,mesh)

    base_name = os.path.basename(mesh_path1).split(".")[0]
    base_names.append(base_name)

    mesh_id, scene1, scene2, _,  obj_id = base_name.split(".")[0].split("_")
    
    # pdb.set_trace()
    # 加载两个mesh文件
    # mesh1 = load_mesh(mesh_path1)
    mesh1 = load_mesh(mesh_path1)

    # mesh_path2 = os.path.join(gt_root,base_name, "model_normalized_manifoldplus.obj")
    mesh_path2 = os.path.join(gt_root,base_name,"model_canonical_manifoldplus.obj")
    mesh2 = load_mesh(mesh_path2)
    
    sdf_path = os.path.join(sdf_root,label2id[label],base_name+".npz")
    sdf = np.load(sdf_path)
    translation_mesh2sdf = sdf['translation_mesh2sdf']
    scale_mesh2sdf = sdf["scale_mesh2sdf"]

    mesh1 = mesh_apply_rts(mesh1,mesh_name="recon",mesh_color="rgba(22,244,244,0.5)")
    # mesh2 = mesh_apply_rts(mesh2,translation_c2w=translation_mesh2sdf,scale_c2w=scale_mesh2sdf,mesh_name="gt")
    mesh2, scale, translation = BoundingCubeNormalization(mesh2)
    # mesh2 = mesh_apply_rts(mesh2,mesh_name="gt")

    # input()

    print(mesh_path1)
    print(mesh_path2)

    # vis_data = [mesh1,mesh2]

    # layout = go.Layout(scene=dict(
    #         aspectmode='data',  # Set the aspect ratio to 'cube' for equal scales
    #         xaxis_title='X',
    #         yaxis_title='Y',
    #         zaxis_title='Z',
    #     ),)

    # fig = go.Figure()
    # fig = go.Figure(data=vis_data, layout=layout)
    # fig.show()

    # 计算IoU
    mesh1 = plotly_mesh3d_to_trimesh(mesh1)
    mesh2 = plotly_mesh3d_to_trimesh(mesh2)
    iou = compute_iou(mesh1, mesh2)
    print(f"IoU: {iou}")
    ious.append(iou)

    # input()

0it [00:00, ?it/s]

/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/13999/test/mesh/Meshes/f98615e51a596fc5c05575120a46cd3b_scene0394_00_ins_8.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/04256520/f98615e51a596fc5c05575120a46cd3b_scene0394_00_ins_8/model_canonical_manifoldplus.obj


1it [00:00,  1.08it/s]

IoU: 0.8624490637959944
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/13999/test/mesh/Meshes/7ae657b39aa2be68ccd1bcd57588acf8_scene0362_01_ins_3.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/04256520/7ae657b39aa2be68ccd1bcd57588acf8_scene0362_01_ins_3/model_canonical_manifoldplus.obj


2it [00:01,  1.35it/s]

...

IoU: 0.9337401218186829
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/13999/test/mesh/Meshes/fb74336a6192c4787afee304cce81d6f_scene0536_01_ins_3.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/04256520/fb74336a6192c4787afee304cce81d6f_scene0536_01_ins_3/model_canonical_manifoldplus.obj


38it [00:31,  1.27it/s]

IoU: 0.9034810446994928
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/13999/test/mesh/Meshes/58447a958c4af154942bb07caacf4df3_scene0474_03_ins_2.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/04256520/58447a958c4af154942bb07caacf4df3_scene0474_03_ins_2/model_canonical_manifoldplus.obj


39it [00:31,  1.24it/s]

IoU: 0.762504248183327


In [21]:
del iou_path
del mesh_list_path

ious = np.array(ious)
print(ious.mean())
iou_path = output_root.replace("/Meshes","/ious.txt")
np.savetxt(iou_path,ious)

mesh_list_path = output_root.replace("/Meshes","/mesh_list.txt")
with open(mesh_list_path,"w") as f:
    for imesh in mesh_list:
        f.write(imesh+"\n")

0.871586262492169


### 9999

In [22]:
label = "sofa"
gt_root = "/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus"
gt_root = os.path.join(gt_root,label2id[label])

sdf_root = "/storage/user/huju/transferred/ws_dditnach/DeepSDF/data/SdfSamples/canonical_mesh_manifoldplus" # 加载相关的transformation

# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/stage2_diff_cond_scanarcw/recon/49999/Meshes"
# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond/recon/23999/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_neighbor/output/49999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_noneighbor/output/69999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/13999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/9999/test/mesh/Meshes"

mesh_list = os.listdir(output_root)
ious = []
base_names = []

for id_mesh, mesh in tqdm(enumerate(mesh_list)):

    mesh_path1 = os.path.join(output_root,mesh)

    base_name = os.path.basename(mesh_path1).split(".")[0]
    base_names.append(base_name)

    mesh_id, scene1, scene2, _,  obj_id = base_name.split(".")[0].split("_")
    
    # pdb.set_trace()
    # 加载两个mesh文件
    # mesh1 = load_mesh(mesh_path1)
    mesh1 = load_mesh(mesh_path1)

    # mesh_path2 = os.path.join(gt_root,base_name, "model_normalized_manifoldplus.obj")
    mesh_path2 = os.path.join(gt_root,base_name,"model_canonical_manifoldplus.obj")
    mesh2 = load_mesh(mesh_path2)
    
    sdf_path = os.path.join(sdf_root,label2id[label],base_name+".npz")
    sdf = np.load(sdf_path)
    translation_mesh2sdf = sdf['translation_mesh2sdf']
    scale_mesh2sdf = sdf["scale_mesh2sdf"]

    mesh1 = mesh_apply_rts(mesh1,mesh_name="recon",mesh_color="rgba(22,244,244,0.5)")
    # mesh2 = mesh_apply_rts(mesh2,translation_c2w=translation_mesh2sdf,scale_c2w=scale_mesh2sdf,mesh_name="gt")
    mesh2, scale, translation = BoundingCubeNormalization(mesh2)
    # mesh2 = mesh_apply_rts(mesh2,mesh_name="gt")

    # input()

    print(mesh_path1)
    print(mesh_path2)

    # vis_data = [mesh1,mesh2]

    # layout = go.Layout(scene=dict(
    #         aspectmode='data',  # Set the aspect ratio to 'cube' for equal scales
    #         xaxis_title='X',
    #         yaxis_title='Y',
    #         zaxis_title='Z',
    #     ),)

    # fig = go.Figure()
    # fig = go.Figure(data=vis_data, layout=layout)
    # fig.show()

    # 计算IoU
    mesh1 = plotly_mesh3d_to_trimesh(mesh1)
    mesh2 = plotly_mesh3d_to_trimesh(mesh2)
    iou = compute_iou(mesh1, mesh2)
    print(f"IoU: {iou}")
    ious.append(iou)

    # input()

ious = np.array(ious)
print(ious.mean())
iou_path = output_root.replace("/Meshes","/ious.txt")
np.savetxt(iou_path,ious)

mesh_list_path = output_root.replace("/Meshes","/mesh_list.txt")
with open(mesh_list_path,"w") as f:
    for imesh in mesh_list:
        f.write(imesh+"\n")

0it [00:00, ?it/s]

/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/9999/test/mesh/Meshes/f98615e51a596fc5c05575120a46cd3b_scene0394_00_ins_8.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/04256520/f98615e51a596fc5c05575120a46cd3b_scene0394_00_ins_8/model_canonical_manifoldplus.obj


1it [00:00,  1.75it/s]

IoU: 0.8565669425719455
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/9999/test/mesh/Meshes/7ae657b39aa2be68ccd1bcd57588acf8_scene0362_01_ins_3.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/04256520/7ae657b39aa2be68ccd1bcd57588acf8_scene0362_01_ins_3/model_canonical_manifoldplus.obj


2it [00:01,  1.65it/s]

...

IoU: 0.9276532651717033
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/9999/test/mesh/Meshes/fb74336a6192c4787afee304cce81d6f_scene0536_01_ins_3.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/04256520/fb74336a6192c4787afee304cce81d6f_scene0536_01_ins_3/model_canonical_manifoldplus.obj


38it [00:31,  1.23it/s]

IoU: 0.9156013705179062
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/9999/test/mesh/Meshes/58447a958c4af154942bb07caacf4df3_scene0474_03_ins_2.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/04256520/58447a958c4af154942bb07caacf4df3_scene0474_03_ins_2/model_canonical_manifoldplus.obj


39it [00:31,  1.23it/s]

IoU: 0.7791076371217477
0.8643563917784155


### 999

In [3]:
label = "sofa"
gt_root = "/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus"
gt_root = os.path.join(gt_root,label2id[label])

sdf_root = "/storage/user/huju/transferred/ws_dditnach/DeepSDF/data/SdfSamples/canonical_mesh_manifoldplus" # 加载相关的transformation

# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/stage2_diff_cond_scanarcw/recon/49999/Meshes"
# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond/recon/23999/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_neighbor/output/49999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_noneighbor/output/69999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/13999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/9999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/999/test/mesh/Meshes"

mesh_list = os.listdir(output_root)
ious = []
base_names = []

for id_mesh, mesh in tqdm(enumerate(mesh_list)):

    mesh_path1 = os.path.join(output_root,mesh)

    base_name = os.path.basename(mesh_path1).split(".")[0]
    base_names.append(base_name)

    mesh_id, scene1, scene2, _,  obj_id = base_name.split(".")[0].split("_")
    
    # pdb.set_trace()
    # 加载两个mesh文件
    # mesh1 = load_mesh(mesh_path1)
    mesh1 = load_mesh(mesh_path1)

    # mesh_path2 = os.path.join(gt_root,base_name, "model_normalized_manifoldplus.obj")
    mesh_path2 = os.path.join(gt_root,base_name,"model_canonical_manifoldplus.obj")
    mesh2 = load_mesh(mesh_path2)
    
    sdf_path = os.path.join(sdf_root,label2id[label],base_name+".npz")
    sdf = np.load(sdf_path)
    translation_mesh2sdf = sdf['translation_mesh2sdf']
    scale_mesh2sdf = sdf["scale_mesh2sdf"]

    mesh1 = mesh_apply_rts(mesh1,mesh_name="recon",mesh_color="rgba(22,244,244,0.5)")
    # mesh2 = mesh_apply_rts(mesh2,translation_c2w=translation_mesh2sdf,scale_c2w=scale_mesh2sdf,mesh_name="gt")
    mesh2, scale, translation = BoundingCubeNormalization(mesh2)
    # mesh2 = mesh_apply_rts(mesh2,mesh_name="gt")

    # input()

    print(mesh_path1)
    print(mesh_path2)

    # vis_data = [mesh1,mesh2]

    # layout = go.Layout(scene=dict(
    #         aspectmode='data',  # Set the aspect ratio to 'cube' for equal scales
    #         xaxis_title='X',
    #         yaxis_title='Y',
    #         zaxis_title='Z',
    #     ),)

    # fig = go.Figure()
    # fig = go.Figure(data=vis_data, layout=layout)
    # fig.show()

    # 计算IoU
    mesh1 = plotly_mesh3d_to_trimesh(mesh1)
    mesh2 = plotly_mesh3d_to_trimesh(mesh2)
    iou = compute_iou(mesh1, mesh2)
    print(f"IoU: {iou}")
    ious.append(iou)

    # input()

del iou_path
del mesh_list_path

ious = np.array(ious)
print(ious.mean())
iou_path = output_root.replace("/Meshes","/ious.txt")
np.savetxt(iou_path,ious)

mesh_list_path = output_root.replace("/Meshes","/mesh_list.txt")
with open(mesh_list_path,"w") as f:
    for imesh in mesh_list:
        f.write(imesh+"\n")

0it [00:00, ?it/s]

/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/999/test/mesh/Meshes/f98615e51a596fc5c05575120a46cd3b_scene0394_00_ins_8.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/04256520/f98615e51a596fc5c05575120a46cd3b_scene0394_00_ins_8/model_canonical_manifoldplus.obj


1it [00:01,  1.63s/it]

IoU: 0.6340679082616474
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/999/test/mesh/Meshes/7ae657b39aa2be68ccd1bcd57588acf8_scene0362_01_ins_3.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/04256520/7ae657b39aa2be68ccd1bcd57588acf8_scene0362_01_ins_3/model_canonical_manifoldplus.obj


2it [00:02,  1.14s/it]

...

IoU: 0.9379779588791678
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/999/test/mesh/Meshes/fb74336a6192c4787afee304cce81d6f_scene0536_01_ins_3.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/04256520/fb74336a6192c4787afee304cce81d6f_scene0536_01_ins_3/model_canonical_manifoldplus.obj


38it [00:39,  1.05it/s]

IoU: 0.9253166329068132
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/999/test/mesh/Meshes/58447a958c4af154942bb07caacf4df3_scene0474_03_ins_2.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/04256520/58447a958c4af154942bb07caacf4df3_scene0474_03_ins_2/model_canonical_manifoldplus.obj


39it [00:39,  1.02s/it]

IoU: 0.7698683045606304
0.8322059445567823


In [ ]:
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa_pcd1000test/output/999/test/mesh

## ddit_stage2_sofa_pcd1000test

### 999

In [5]:
label = "sofa"
gt_root = "/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus"
gt_root = os.path.join(gt_root,label2id[label])

sdf_root = "/storage/user/huju/transferred/ws_dditnach/DeepSDF/data/SdfSamples/canonical_mesh_manifoldplus" # 加载相关的transformation

# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/stage2_diff_cond_scanarcw/recon/49999/Meshes"
# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond/recon/23999/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_neighbor/output/49999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_noneighbor/output/69999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/13999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/9999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa_pcd1000test/output/999/test/mesh/Meshes"

mesh_list = os.listdir(output_root)
ious = []
base_names = []

for id_mesh, mesh in tqdm(enumerate(mesh_list)):

    mesh_path1 = os.path.join(output_root,mesh)

    base_name = os.path.basename(mesh_path1).split(".")[0]
    base_names.append(base_name)

    mesh_id, scene1, scene2, _,  obj_id = base_name.split(".")[0].split("_")
    
    # pdb.set_trace()
    # 加载两个mesh文件
    # mesh1 = load_mesh(mesh_path1)
    mesh1 = load_mesh(mesh_path1)

    # mesh_path2 = os.path.join(gt_root,base_name, "model_normalized_manifoldplus.obj")
    mesh_path2 = os.path.join(gt_root,base_name,"model_canonical_manifoldplus.obj")
    mesh2 = load_mesh(mesh_path2)
    
    sdf_path = os.path.join(sdf_root,label2id[label],base_name+".npz")
    sdf = np.load(sdf_path)
    translation_mesh2sdf = sdf['translation_mesh2sdf']
    scale_mesh2sdf = sdf["scale_mesh2sdf"]

    mesh1 = mesh_apply_rts(mesh1,mesh_name="recon",mesh_color="rgba(22,244,244,0.5)")
    # mesh2 = mesh_apply_rts(mesh2,translation_c2w=translation_mesh2sdf,scale_c2w=scale_mesh2sdf,mesh_name="gt")
    mesh2, scale, translation = BoundingCubeNormalization(mesh2)
    # mesh2 = mesh_apply_rts(mesh2,mesh_name="gt")

    # input()

    print(mesh_path1)
    print(mesh_path2)

    # vis_data = [mesh1,mesh2]

    # layout = go.Layout(scene=dict(
    #         aspectmode='data',  # Set the aspect ratio to 'cube' for equal scales
    #         xaxis_title='X',
    #         yaxis_title='Y',
    #         zaxis_title='Z',
    #     ),)

    # fig = go.Figure()
    # fig = go.Figure(data=vis_data, layout=layout)
    # fig.show()

    # 计算IoU
    mesh1 = plotly_mesh3d_to_trimesh(mesh1)
    mesh2 = plotly_mesh3d_to_trimesh(mesh2)
    iou = compute_iou(mesh1, mesh2)
    print(f"IoU: {iou}")
    ious.append(iou)

    # input()

del iou_path
del mesh_list_path

ious = np.array(ious)
print(ious.mean())
iou_path = output_root.replace("/Meshes","/ious.txt")
np.savetxt(iou_path,ious)

mesh_list_path = output_root.replace("/Meshes","/mesh_list.txt")
with open(mesh_list_path,"w") as f:
    for imesh in mesh_list:
        f.write(imesh+"\n")

0it [00:00, ?it/s]

/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa_pcd1000test/output/999/test/mesh/Meshes/f98615e51a596fc5c05575120a46cd3b_scene0394_00_ins_8.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/04256520/f98615e51a596fc5c05575120a46cd3b_scene0394_00_ins_8/model_canonical_manifoldplus.obj


1it [00:00,  1.09it/s]

IoU: 0.6157604842439975
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa_pcd1000test/output/999/test/mesh/Meshes/7ae657b39aa2be68ccd1bcd57588acf8_scene0362_01_ins_3.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/04256520/7ae657b39aa2be68ccd1bcd57588acf8_scene0362_01_ins_3/model_canonical_manifoldplus.obj


2it [00:01,  1.10it/s]

...

IoU: 0.8654448876818512
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa_pcd1000test/output/999/test/mesh/Meshes/fb74336a6192c4787afee304cce81d6f_scene0536_01_ins_3.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/04256520/fb74336a6192c4787afee304cce81d6f_scene0536_01_ins_3/model_canonical_manifoldplus.obj


38it [00:31,  1.33it/s]

IoU: 0.902535237824021
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa_pcd1000test/output/999/test/mesh/Meshes/58447a958c4af154942bb07caacf4df3_scene0474_03_ins_2.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/04256520/58447a958c4af154942bb07caacf4df3_scene0474_03_ins_2/model_canonical_manifoldplus.obj


39it [00:31,  1.22it/s]

IoU: 0.7438844152210969
0.802934488440467


## dimr_stage2_sofa

In [ ]:
label = "sofa"
gt_root = "/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus"
gt_root = os.path.join(gt_root,label2id[label])

sdf_root = "/storage/user/huju/transferred/ws_dditnach/DeepSDF/data/SdfSamples/canonical_mesh_manifoldplus" # 加载相关的transformation

# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/stage2_diff_cond_scanarcw/recon/49999/Meshes"
# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond/recon/23999/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_neighbor/output/49999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_noneighbor/output/69999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/13999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/9999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/dimr_stage2_sofa/output/1199/test/mesh/Meshes"

mesh_list = os.listdir(output_root)
ious = []
base_names = []

for id_mesh, mesh in tqdm(enumerate(mesh_list)):

    mesh_path1 = os.path.join(output_root,mesh)

    base_name = os.path.basename(mesh_path1).split(".")[0]
    base_names.append(base_name)

    mesh_id, scene1, scene2, _,  obj_id = base_name.split(".")[0].split("_")
    
    # pdb.set_trace()
    # 加载两个mesh文件
    # mesh1 = load_mesh(mesh_path1)
    mesh1 = load_mesh(mesh_path1)

    # mesh_path2 = os.path.join(gt_root,base_name, "model_normalized_manifoldplus.obj")
    mesh_path2 = os.path.join(gt_root,base_name,"model_canonical_manifoldplus.obj")
    mesh2 = load_mesh(mesh_path2)
    
    sdf_path = os.path.join(sdf_root,label2id[label],base_name+".npz")
    sdf = np.load(sdf_path)
    translation_mesh2sdf = sdf['translation_mesh2sdf']
    scale_mesh2sdf = sdf["scale_mesh2sdf"]

    mesh1 = mesh_apply_rts(mesh1,mesh_name="recon",mesh_color="rgba(22,244,244,0.5)")
    # mesh2 = mesh_apply_rts(mesh2,translation_c2w=translation_mesh2sdf,scale_c2w=scale_mesh2sdf,mesh_name="gt")
    mesh2, scale, translation = BoundingCubeNormalization(mesh2)
    # mesh2 = mesh_apply_rts(mesh2,mesh_name="gt")

    # input()

    print(mesh_path1)
    print(mesh_path2)

    # vis_data = [mesh1,mesh2]

    # layout = go.Layout(scene=dict(
    #         aspectmode='data',  # Set the aspect ratio to 'cube' for equal scales
    #         xaxis_title='X',
    #         yaxis_title='Y',
    #         zaxis_title='Z',
    #     ),)

    # fig = go.Figure()
    # fig = go.Figure(data=vis_data, layout=layout)
    # fig.show()

    # 计算IoU
    mesh1 = plotly_mesh3d_to_trimesh(mesh1)
    mesh2 = plotly_mesh3d_to_trimesh(mesh2)
    iou = compute_iou(mesh1, mesh2)
    print(f"IoU: {iou}")
    ious.append(iou)

    # input()

del iou_path
del mesh_list_path

ious = np.array(ious)
print(ious.mean())
iou_path = output_root.replace("/Meshes","/ious.txt")
np.savetxt(iou_path,ious)

mesh_list_path = output_root.replace("/Meshes","/mesh_list.txt")
with open(mesh_list_path,"w") as f:
    for imesh in mesh_list:
        f.write(imesh+"\n")

0it [00:00, ?it/s]

/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/dimr_stage2_sofa/output/1199/test/mesh/Meshes/f98615e51a596fc5c05575120a46cd3b_scene0394_00_ins_8.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/04256520/f98615e51a596fc5c05575120a46cd3b_scene0394_00_ins_8/model_canonical_manifoldplus.obj


1it [00:00,  1.53it/s]

IoU: 0.4773429399855722
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/dimr_stage2_sofa/output/1199/test/mesh/Meshes/7ae657b39aa2be68ccd1bcd57588acf8_scene0362_01_ins_3.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/04256520/7ae657b39aa2be68ccd1bcd57588acf8_scene0362_01_ins_3/model_canonical_manifoldplus.obj


2it [00:01,  1.53it/s]

...

IoU: 0.5743137515345503
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/dimr_stage2_sofa/output/1199/test/mesh/Meshes/fb74336a6192c4787afee304cce81d6f_scene0536_01_ins_3.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/04256520/fb74336a6192c4787afee304cce81d6f_scene0536_01_ins_3/model_canonical_manifoldplus.obj


38it [00:33,  1.05it/s]

IoU: 0.795698722098155
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/dimr_stage2_sofa/output/1199/test/mesh/Meshes/58447a958c4af154942bb07caacf4df3_scene0474_03_ins_2.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/04256520/58447a958c4af154942bb07caacf4df3_scene0474_03_ins_2/model_canonical_manifoldplus.obj


39it [00:34,  1.13it/s]

IoU: 0.39538546299368
0.5599778597661367


# Chair

## ddit_stage2_diff_cond_chair_train_neighbor

In [2]:
label = "chair"
gt_root = "/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus"
gt_root = os.path.join(gt_root,label2id[label])

sdf_root = "/storage/user/huju/transferred/ws_dditnach/DeepSDF/data/SdfSamples/canonical_mesh_manifoldplus" # 加载相关的transformation

# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/stage2_diff_cond_scanarcw/recon/49999/Meshes"
# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond/recon/23999/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_neighbor/output/49999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_noneighbor/output/69999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/13999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/9999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/dimr_stage2_sofa/output/1199/test/mesh/Meshes"

output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_chair_train_neighbor/output/18999/test/mesh/Meshes"

mesh_list = os.listdir(output_root)
ious = []
base_names = []

for id_mesh, mesh in tqdm(enumerate(mesh_list)):

    mesh_path1 = os.path.join(output_root,mesh)

    base_name = os.path.basename(mesh_path1).split(".")[0]
    base_names.append(base_name)

    mesh_id, scene1, scene2, _,  obj_id = base_name.split(".")[0].split("_")
    
    # pdb.set_trace()
    # 加载两个mesh文件
    # mesh1 = load_mesh(mesh_path1)
    mesh1 = load_mesh(mesh_path1)

    # mesh_path2 = os.path.join(gt_root,base_name, "model_normalized_manifoldplus.obj")
    mesh_path2 = os.path.join(gt_root,base_name,"model_canonical_manifoldplus.obj")
    mesh2 = load_mesh(mesh_path2)
    
    sdf_path = os.path.join(sdf_root,label2id[label],base_name+".npz")
    sdf = np.load(sdf_path)
    translation_mesh2sdf = sdf['translation_mesh2sdf']
    scale_mesh2sdf = sdf["scale_mesh2sdf"]

    mesh1 = mesh_apply_rts(mesh1,mesh_name="recon",mesh_color="rgba(22,244,244,0.5)")
    # mesh2 = mesh_apply_rts(mesh2,translation_c2w=translation_mesh2sdf,scale_c2w=scale_mesh2sdf,mesh_name="gt")
    mesh2, scale, translation = BoundingCubeNormalization(mesh2)
    # mesh2 = mesh_apply_rts(mesh2,mesh_name="gt")

    # input()

    print(mesh_path1)
    print(mesh_path2)

    # vis_data = [mesh1,mesh2]

    # layout = go.Layout(scene=dict(
    #         aspectmode='data',  # Set the aspect ratio to 'cube' for equal scales
    #         xaxis_title='X',
    #         yaxis_title='Y',
    #         zaxis_title='Z',
    #     ),)

    # fig = go.Figure()
    # fig = go.Figure(data=vis_data, layout=layout)
    # fig.show()

    # 计算IoU
    mesh1 = plotly_mesh3d_to_trimesh(mesh1)
    mesh2 = plotly_mesh3d_to_trimesh(mesh2)
    iou = compute_iou(mesh1, mesh2)
    print(f"IoU: {iou}")
    ious.append(iou)

    # input()

del iou_path
del mesh_list_path

0it [00:00, ?it/s]

/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_chair_train_neighbor/output/18999/test/mesh/Meshes/7e8b24aab1f2681e595557081060d0b_scene0378_01_ins_6.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/03001627/7e8b24aab1f2681e595557081060d0b_scene0378_01_ins_6/model_canonical_manifoldplus.obj


1it [00:05,  5.99s/it]

IoU: 0.7728223109378027
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_chair_train_neighbor/output/18999/test/mesh/Meshes/403b4eb7194f563f79b3e1be3524f72f_scene0024_01_ins_7.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/03001627/403b4eb7194f563f79b3e1be3524f72f_scene0024_01_ins_7/model_canonical_manifoldplus.obj


2it [00:08,  4.05s/it]

...

281it [12:21,  1.79s/it]

IoU: 0.5302895530934749
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_chair_train_neighbor/output/18999/test/mesh/Meshes/8c629a89e570c8776a9cd58b3a6e8ee5_scene0627_01_ins_4.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/03001627/8c629a89e570c8776a9cd58b3a6e8ee5_scene0627_01_ins_4/model_canonical_manifoldplus.obj


282it [12:24,  2.64s/it]

IoU: 0.6373651221268383


NameError: name 'iou_path' is not defined

In [5]:
ious = np.array(ious)
print(ious.mean())
iou_path = output_root.replace("/Meshes","/ious.txt")
print(iou_path)
np.savetxt(iou_path,ious)

mesh_list_path = output_root.replace("/Meshes","/mesh_list.txt")
print(mesh_list_path)
with open(mesh_list_path,"w") as f:
    for imesh in mesh_list:
        f.write(imesh+"\n")

0.5769165370015834
0.5769165370015834
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_chair_train_neighbor/output/18999/test/mesh/ious.txt
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_chair_train_neighbor/output/18999/test/mesh/mesh_list.txt


## ddit_stage2_diff_cond_chair_train_noneighbor

In [6]:
label = "chair"
gt_root = "/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus"
gt_root = os.path.join(gt_root,label2id[label])

sdf_root = "/storage/user/huju/transferred/ws_dditnach/DeepSDF/data/SdfSamples/canonical_mesh_manifoldplus" # 加载相关的transformation

# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/stage2_diff_cond_scanarcw/recon/49999/Meshes"
# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond/recon/23999/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_neighbor/output/49999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_noneighbor/output/69999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/13999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/9999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/dimr_stage2_sofa/output/1199/test/mesh/Meshes"

output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_chair_train_noneighbor/output/18999/test/mesh/oldMeshes"

mesh_list = os.listdir(output_root)
ious = []
base_names = []

for id_mesh, mesh in tqdm(enumerate(mesh_list)):

    mesh_path1 = os.path.join(output_root,mesh)

    base_name = os.path.basename(mesh_path1).split(".")[0]
    base_names.append(base_name)

    mesh_id, scene1, scene2, _,  obj_id = base_name.split(".")[0].split("_")
    
    # pdb.set_trace()
    # 加载两个mesh文件
    # mesh1 = load_mesh(mesh_path1)
    mesh1 = load_mesh(mesh_path1)

    # mesh_path2 = os.path.join(gt_root,base_name, "model_normalized_manifoldplus.obj")
    mesh_path2 = os.path.join(gt_root,base_name,"model_canonical_manifoldplus.obj")
    mesh2 = load_mesh(mesh_path2)
    
    sdf_path = os.path.join(sdf_root,label2id[label],base_name+".npz")
    sdf = np.load(sdf_path)
    translation_mesh2sdf = sdf['translation_mesh2sdf']
    scale_mesh2sdf = sdf["scale_mesh2sdf"]

    mesh1 = mesh_apply_rts(mesh1,mesh_name="recon",mesh_color="rgba(22,244,244,0.5)")
    # mesh2 = mesh_apply_rts(mesh2,translation_c2w=translation_mesh2sdf,scale_c2w=scale_mesh2sdf,mesh_name="gt")
    mesh2, scale, translation = BoundingCubeNormalization(mesh2)
    # mesh2 = mesh_apply_rts(mesh2,mesh_name="gt")

    # input()

    print(mesh_path1)
    print(mesh_path2)

    # vis_data = [mesh1,mesh2]

    # layout = go.Layout(scene=dict(
    #         aspectmode='data',  # Set the aspect ratio to 'cube' for equal scales
    #         xaxis_title='X',
    #         yaxis_title='Y',
    #         zaxis_title='Z',
    #     ),)

    # fig = go.Figure()
    # fig = go.Figure(data=vis_data, layout=layout)
    # fig.show()

    # 计算IoU
    mesh1 = plotly_mesh3d_to_trimesh(mesh1)
    mesh2 = plotly_mesh3d_to_trimesh(mesh2)
    iou = compute_iou(mesh1, mesh2)
    print(f"IoU: {iou}")
    ious.append(iou)

    # input()

del iou_path
del mesh_list_path

ious = np.array(ious)
print(ious.mean())
iou_path = os.path.join(os.path.dirname(output_root),"/ious.txt")
np.savetxt(iou_path,ious)

mesh_list_path = os.path.join(os.path.dirname(output_root),"/mesh_list.txt")
with open(mesh_list_path,"w") as f:
    for imesh in mesh_list:
        f.write(imesh+"\n")

0it [00:00, ?it/s]

/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_chair_train_noneighbor/output/18999/test/mesh/oldMeshes/7e8b24aab1f2681e595557081060d0b_scene0378_01_ins_6.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/03001627/7e8b24aab1f2681e595557081060d0b_scene0378_01_ins_6/model_canonical_manifoldplus.obj


1it [00:02,  2.23s/it]

IoU: 0.7905813440933144
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_chair_train_noneighbor/output/18999/test/mesh/oldMeshes/403b4eb7194f563f79b3e1be3524f72f_scene0024_01_ins_7.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/03001627/403b4eb7194f563f79b3e1be3524f72f_scene0024_01_ins_7/model_canonical_manifoldplus.obj


2it [00:04,  1.97s/it]

...

383it [12:36,  1.58s/it]

IoU: 0.8506372898597243
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_chair_train_noneighbor/output/18999/test/mesh/oldMeshes/8c629a89e570c8776a9cd58b3a6e8ee5_scene0627_01_ins_4.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/03001627/8c629a89e570c8776a9cd58b3a6e8ee5_scene0627_01_ins_4/model_canonical_manifoldplus.obj


384it [12:39,  1.98s/it]

IoU: 0.8463704658960565
0.7051174147580909


PermissionError: [Errno 13] Permission denied: '/ious.txt'

In [8]:
# del iou_path
# del mesh_list_path

ious = np.array(ious)
print(ious.mean())
iou_path = os.path.join(os.path.dirname(output_root),"ious.txt")
np.savetxt(iou_path,ious)

mesh_list_path = os.path.join(os.path.dirname(output_root),"mesh_list.txt")
with open(mesh_list_path,"w") as f:
    for imesh in mesh_list:
        f.write(imesh+"\n")

0.7051174147580909


## ddit_stage2_chair

In [11]:
label = "chair"
gt_root = "/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus"
gt_root = os.path.join(gt_root,label2id[label])

sdf_root = "/storage/user/huju/transferred/ws_dditnach/DeepSDF/data/SdfSamples/canonical_mesh_manifoldplus" # 加载相关的transformation

# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/stage2_diff_cond_scanarcw/recon/49999/Meshes"
# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond/recon/23999/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_neighbor/output/49999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_noneighbor/output/69999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/13999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/9999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/dimr_stage2_sofa/output/1199/test/mesh/Meshes"

output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_chair_train_noneighbor/output/18999/test/mesh/oldMeshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_chair/output/999/test/mesh/Meshes"

mesh_list = os.listdir(output_root)
ious = []
base_names = []

for id_mesh, mesh in tqdm(enumerate(mesh_list)):

    mesh_path1 = os.path.join(output_root,mesh)

    base_name = os.path.basename(mesh_path1).split(".")[0]
    base_names.append(base_name)

    mesh_id, scene1, scene2, _,  obj_id = base_name.split(".")[0].split("_")
    
    # pdb.set_trace()
    # 加载两个mesh文件
    # mesh1 = load_mesh(mesh_path1)
    mesh1 = load_mesh(mesh_path1)

    # mesh_path2 = os.path.join(gt_root,base_name, "model_normalized_manifoldplus.obj")
    mesh_path2 = os.path.join(gt_root,base_name,"model_canonical_manifoldplus.obj")
    mesh2 = load_mesh(mesh_path2)
    
    sdf_path = os.path.join(sdf_root,label2id[label],base_name+".npz")
    sdf = np.load(sdf_path)
    translation_mesh2sdf = sdf['translation_mesh2sdf']
    scale_mesh2sdf = sdf["scale_mesh2sdf"]

    mesh1 = mesh_apply_rts(mesh1,mesh_name="recon",mesh_color="rgba(22,244,244,0.5)")
    # mesh2 = mesh_apply_rts(mesh2,translation_c2w=translation_mesh2sdf,scale_c2w=scale_mesh2sdf,mesh_name="gt")
    mesh2, scale, translation = BoundingCubeNormalization(mesh2)
    # mesh2 = mesh_apply_rts(mesh2,mesh_name="gt")

    # input()

    print(mesh_path1)
    print(mesh_path2)

    # vis_data = [mesh1,mesh2]

    # layout = go.Layout(scene=dict(
    #         aspectmode='data',  # Set the aspect ratio to 'cube' for equal scales
    #         xaxis_title='X',
    #         yaxis_title='Y',
    #         zaxis_title='Z',
    #     ),)

    # fig = go.Figure()
    # fig = go.Figure(data=vis_data, layout=layout)
    # fig.show()

    # 计算IoU
    mesh1 = plotly_mesh3d_to_trimesh(mesh1)
    mesh2 = plotly_mesh3d_to_trimesh(mesh2)
    iou = compute_iou(mesh1, mesh2)
    print(f"IoU: {iou}")
    ious.append(iou)

    # input()

0it [00:00, ?it/s]

/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_chair/output/999/test/mesh/Meshes/7e8b24aab1f2681e595557081060d0b_scene0378_01_ins_6.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/03001627/7e8b24aab1f2681e595557081060d0b_scene0378_01_ins_6/model_canonical_manifoldplus.obj


1it [00:02,  2.28s/it]

IoU: 0.8612480772941464
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_chair/output/999/test/mesh/Meshes/403b4eb7194f563f79b3e1be3524f72f_scene0024_01_ins_7.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/03001627/403b4eb7194f563f79b3e1be3524f72f_scene0024_01_ins_7/model_canonical_manifoldplus.obj


2it [00:03,  1.91s/it]

...

IoU: 0.8307602532621263
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_chair/output/999/test/mesh/Meshes/8ab6783b1dfbf3a8a5d9ad16964840ab_scene0529_01_ins_9.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/03001627/8ab6783b1dfbf3a8a5d9ad16964840ab_scene0529_01_ins_9/model_canonical_manifoldplus.obj


383it [11:56,  1.55s/it]

IoU: 0.8635832177177871
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_chair/output/999/test/mesh/Meshes/8c629a89e570c8776a9cd58b3a6e8ee5_scene0627_01_ins_4.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/03001627/8c629a89e570c8776a9cd58b3a6e8ee5_scene0627_01_ins_4/model_canonical_manifoldplus.obj


384it [11:59,  1.87s/it]

IoU: 0.8724651835863326


In [12]:
del iou_path
del mesh_list_path

ious = np.array(ious)
print(ious.mean())
iou_path = os.path.join(os.path.dirname(output_root),"ious.txt")
np.savetxt(iou_path, ious)

mesh_list_path = os.path.join(os.path.dirname(output_root),"mesh_list.txt")
with open(mesh_list_path,"w") as f:
    for imesh in mesh_list:
        f.write(imesh+"\n")

0.820983499343614


# Table

## ddit_stage2_table

In [4]:
label = "table"
gt_root = "/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus"
gt_root = os.path.join(gt_root,label2id[label])

sdf_root = "/storage/user/huju/transferred/ws_dditnach/DeepSDF/data/SdfSamples/canonical_mesh_manifoldplus" # 加载相关的transformation

# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/stage2_diff_cond_scanarcw/recon/49999/Meshes"
# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond/recon/23999/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_neighbor/output/49999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_noneighbor/output/69999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/13999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/9999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/dimr_stage2_sofa/output/1199/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_chair_train_noneighbor/output/18999/test/mesh/oldMeshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_chair/output/999/test/mesh/Meshes"

output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_table/output/999/test/mesh/Meshes"

mesh_list = os.listdir(output_root)
ious = []
base_names = []

for id_mesh, mesh in tqdm(enumerate(mesh_list)):

    mesh_path1 = os.path.join(output_root,mesh)

    base_name = os.path.basename(mesh_path1).split(".")[0]
    base_names.append(base_name)

    mesh_id, scene1, scene2, _,  obj_id = base_name.split(".")[0].split("_")
    
    # pdb.set_trace()
    # 加载两个mesh文件
    # mesh1 = load_mesh(mesh_path1)
    mesh1 = load_mesh(mesh_path1)

    # mesh_path2 = os.path.join(gt_root,base_name, "model_normalized_manifoldplus.obj")
    mesh_path2 = os.path.join(gt_root,base_name,"model_canonical_manifoldplus.obj")
    mesh2 = load_mesh(mesh_path2)
    
    sdf_path = os.path.join(sdf_root,label2id[label],base_name+".npz")
    sdf = np.load(sdf_path)
    translation_mesh2sdf = sdf['translation_mesh2sdf']
    scale_mesh2sdf = sdf["scale_mesh2sdf"]

    mesh1 = mesh_apply_rts(mesh1,mesh_name="recon",mesh_color="rgba(22,244,244,0.5)")
    # mesh2 = mesh_apply_rts(mesh2,translation_c2w=translation_mesh2sdf,scale_c2w=scale_mesh2sdf,mesh_name="gt")
    mesh2, scale, translation = BoundingCubeNormalization(mesh2)
    # mesh2 = mesh_apply_rts(mesh2,mesh_name="gt")

    # input()

    print(mesh_path1)
    print(mesh_path2)

    # vis_data = [mesh1,mesh2]

    # layout = go.Layout(scene=dict(
    #         aspectmode='data',  # Set the aspect ratio to 'cube' for equal scales
    #         xaxis_title='X',
    #         yaxis_title='Y',
    #         zaxis_title='Z',
    #     ),)

    # fig = go.Figure()
    # fig = go.Figure(data=vis_data, layout=layout)
    # fig.show()

    # 计算IoU
    mesh1 = plotly_mesh3d_to_trimesh(mesh1)
    mesh2 = plotly_mesh3d_to_trimesh(mesh2)
    iou = compute_iou(mesh1, mesh2)
    print(f"IoU: {iou}")
    ious.append(iou)

    # input()

0it [00:00, ?it/s]

/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_table/output/999/test/mesh/Meshes/bcfda4ca76d65ca8d4eb6d6477feba27_scene0078_00_ins_2.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/04379243/bcfda4ca76d65ca8d4eb6d6477feba27_scene0078_00_ins_2/model_canonical_manifoldplus.obj


1it [00:02,  2.29s/it]

IoU: 0.7563182876277371
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_table/output/999/test/mesh/Meshes/899f815eaad62b56d8cad143689f8b51_scene0372_00_ins_1.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/04379243/899f815eaad62b56d8cad143689f8b51_scene0372_00_ins_1/model_canonical_manifoldplus.obj


2it [00:04,  2.03s/it]

...

IoU: 0.8918528146667333
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_table/output/999/test/mesh/Meshes/e98dcd0334fa639fef9f898cda0ff409_scene0221_01_ins_7.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/04379243/e98dcd0334fa639fef9f898cda0ff409_scene0221_01_ins_7/model_canonical_manifoldplus.obj


226it [07:50,  2.94s/it]

IoU: 0.882742755804108
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_table/output/999/test/mesh/Meshes/46a07944ca646df01ff8bce0e3352d0c_scene0426_03_ins_5.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/04379243/46a07944ca646df01ff8bce0e3352d0c_scene0426_03_ins_5/model_canonical_manifoldplus.obj


227it [07:51,  2.08s/it]

IoU: 0.9024325101775656


In [7]:
try:
    del iou_path
    del mesh_list_path
except:
    pass

ious = np.array(ious)
print(ious.mean())
iou_path = os.path.join(os.path.dirname(output_root),"ious.txt")
np.savetxt(iou_path, ious)

mesh_list_path = os.path.join(os.path.dirname(output_root),"mesh_list.txt")
with open(mesh_list_path,"w") as f:
    for imesh in mesh_list:
        f.write(imesh+"\n")

0.8733523966911917


## ddit_stage2_diff_cond_table_train_noneighbor

In [8]:
label = "table"
gt_root = "/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus"
gt_root = os.path.join(gt_root,label2id[label])

sdf_root = "/storage/user/huju/transferred/ws_dditnach/DeepSDF/data/SdfSamples/canonical_mesh_manifoldplus" # 加载相关的transformation

# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/stage2_diff_cond_scanarcw/recon/49999/Meshes"
# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond/recon/23999/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_neighbor/output/49999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_noneighbor/output/69999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/13999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/9999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/dimr_stage2_sofa/output/1199/test/mesh/Meshes"

output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_chair_train_noneighbor/output/18999/test/mesh/oldMeshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_chair/output/999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_table_train_noneighbor/output/50999/test/mesh/Meshes"

mesh_list = os.listdir(output_root)
ious = []
base_names = []

for id_mesh, mesh in tqdm(enumerate(mesh_list)):

    mesh_path1 = os.path.join(output_root,mesh)

    base_name = os.path.basename(mesh_path1).split(".")[0]
    base_names.append(base_name)

    mesh_id, scene1, scene2, _,  obj_id = base_name.split(".")[0].split("_")
    
    # pdb.set_trace()
    # 加载两个mesh文件
    # mesh1 = load_mesh(mesh_path1)
    mesh1 = load_mesh(mesh_path1)

    # mesh_path2 = os.path.join(gt_root,base_name, "model_normalized_manifoldplus.obj")
    mesh_path2 = os.path.join(gt_root,base_name,"model_canonical_manifoldplus.obj")
    mesh2 = load_mesh(mesh_path2)
    
    sdf_path = os.path.join(sdf_root,label2id[label],base_name+".npz")
    sdf = np.load(sdf_path)
    translation_mesh2sdf = sdf['translation_mesh2sdf']
    scale_mesh2sdf = sdf["scale_mesh2sdf"]

    mesh1 = mesh_apply_rts(mesh1,mesh_name="recon",mesh_color="rgba(22,244,244,0.5)")
    # mesh2 = mesh_apply_rts(mesh2,translation_c2w=translation_mesh2sdf,scale_c2w=scale_mesh2sdf,mesh_name="gt")
    mesh2, scale, translation = BoundingCubeNormalization(mesh2)
    # mesh2 = mesh_apply_rts(mesh2,mesh_name="gt")

    # input()

    print(mesh_path1)
    print(mesh_path2)

    # vis_data = [mesh1,mesh2]

    # layout = go.Layout(scene=dict(
    #         aspectmode='data',  # Set the aspect ratio to 'cube' for equal scales
    #         xaxis_title='X',
    #         yaxis_title='Y',
    #         zaxis_title='Z',
    #     ),)

    # fig = go.Figure()
    # fig = go.Figure(data=vis_data, layout=layout)
    # fig.show()

    # 计算IoU
    mesh1 = plotly_mesh3d_to_trimesh(mesh1)
    mesh2 = plotly_mesh3d_to_trimesh(mesh2)
    iou = compute_iou(mesh1, mesh2)
    print(f"IoU: {iou}")
    ious.append(iou)

    # input()

0it [00:00, ?it/s]

/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_table_train_noneighbor/output/50999/test/mesh/Meshes/bcfda4ca76d65ca8d4eb6d6477feba27_scene0078_00_ins_2.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/04379243/bcfda4ca76d65ca8d4eb6d6477feba27_scene0078_00_ins_2/model_canonical_manifoldplus.obj


1it [00:03,  3.77s/it]

IoU: 0.9406398707318419
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_table_train_noneighbor/output/50999/test/mesh/Meshes/899f815eaad62b56d8cad143689f8b51_scene0372_00_ins_1.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/04379243/899f815eaad62b56d8cad143689f8b51_scene0372_00_ins_1/model_canonical_manifoldplus.obj


2it [00:04,  2.24s/it]

...

IoU: 0.9071660633409275
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_table_train_noneighbor/output/50999/test/mesh/Meshes/e98dcd0334fa639fef9f898cda0ff409_scene0221_01_ins_7.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/04379243/e98dcd0334fa639fef9f898cda0ff409_scene0221_01_ins_7/model_canonical_manifoldplus.obj


226it [09:10,  6.67s/it]

IoU: 0.8770802329959714
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_table_train_noneighbor/output/50999/test/mesh/Meshes/46a07944ca646df01ff8bce0e3352d0c_scene0426_03_ins_5.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/04379243/46a07944ca646df01ff8bce0e3352d0c_scene0426_03_ins_5/model_canonical_manifoldplus.obj


227it [09:12,  2.43s/it]

IoU: 0.8441044296483443


In [9]:
try:
    del iou_path
    del mesh_list_path
except:
    pass

ious = np.array(ious)
print(ious.mean())
iou_path = os.path.join(os.path.dirname(output_root),"ious.txt")
np.savetxt(iou_path, ious)

mesh_list_path = os.path.join(os.path.dirname(output_root),"mesh_list.txt")
with open(mesh_list_path,"w") as f:
    for imesh in mesh_list:
        f.write(imesh+"\n")

0.7119690566770771


# Bed

## ddit_stage2_diff_cond_bed_train_noneighbor

In [4]:
label = "bed"
gt_root = "/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus"
gt_root = os.path.join(gt_root,label2id[label])

sdf_root = "/storage/user/huju/transferred/ws_dditnach/DeepSDF/data/SdfSamples/canonical_mesh_manifoldplus" # 加载相关的transformation

# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/stage2_diff_cond_scanarcw/recon/49999/Meshes"
# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond/recon/23999/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_neighbor/output/49999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_noneighbor/output/69999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/13999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/9999/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/dimr_stage2_sofa/output/1199/test/mesh/Meshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_chair_train_noneighbor/output/18999/test/mesh/oldMeshes"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_chair/output/999/test/mesh/Meshes"

output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_bed_train_noneighbor/output/55999/test/mesh"

mesh_list = os.listdir(output_root)
ious = []
base_names = []

for id_mesh, mesh in tqdm(enumerate(mesh_list)):

    mesh_path1 = os.path.join(output_root,mesh)

    base_name = os.path.basename(mesh_path1).split(".")[0]
    base_names.append(base_name)

    mesh_id, scene1, scene2, _,  obj_id = base_name.split(".")[0].split("_")
    
    # pdb.set_trace()
    # 加载两个mesh文件
    # mesh1 = load_mesh(mesh_path1)
    mesh1 = load_mesh(mesh_path1)

    # mesh_path2 = os.path.join(gt_root,base_name, "model_normalized_manifoldplus.obj")
    mesh_path2 = os.path.join(gt_root,base_name,"model_canonical_manifoldplus.obj")
    mesh2 = load_mesh(mesh_path2)
    
    sdf_path = os.path.join(sdf_root,label2id[label],base_name+".npz")
    sdf = np.load(sdf_path)
    translation_mesh2sdf = sdf['translation_mesh2sdf']
    scale_mesh2sdf = sdf["scale_mesh2sdf"]

    mesh1 = mesh_apply_rts(mesh1,mesh_name="recon",mesh_color="rgba(22,244,244,0.5)")
    # mesh2 = mesh_apply_rts(mesh2,translation_c2w=translation_mesh2sdf,scale_c2w=scale_mesh2sdf,mesh_name="gt")
    mesh2, scale, translation = BoundingCubeNormalization(mesh2)
    # mesh2 = mesh_apply_rts(mesh2,mesh_name="gt")

    # input()

    print(mesh_path1)
    print(mesh_path2)

    # vis_data = [mesh1,mesh2]

    # layout = go.Layout(scene=dict(
    #         aspectmode='data',  # Set the aspect ratio to 'cube' for equal scales
    #         xaxis_title='X',
    #         yaxis_title='Y',
    #         zaxis_title='Z',
    #     ),)

    # fig = go.Figure()
    # fig = go.Figure(data=vis_data, layout=layout)
    # fig.show()

    # 计算IoU
    mesh1 = plotly_mesh3d_to_trimesh(mesh1)
    mesh2 = plotly_mesh3d_to_trimesh(mesh2)
    iou = compute_iou(mesh1, mesh2)
    print(f"IoU: {iou}")
    ious.append(iou)

    # input()

0it [00:00, ?it/s]

/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_bed_train_noneighbor/output/55999/test/mesh/946bd5aeda453b38b3a454ed6a7199e2_scene0426_03_ins_2.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/02818832/946bd5aeda453b38b3a454ed6a7199e2_scene0426_03_ins_2/model_canonical_manifoldplus.obj


1it [00:01,  1.27s/it]

IoU: 0.49339884500197534
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_bed_train_noneighbor/output/55999/test/mesh/51223b1b770ff5e72f38f5bd71072746_scene0376_01_ins_4.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/02818832/51223b1b770ff5e72f38f5bd71072746_scene0376_01_ins_4/model_canonical_manifoldplus.obj


2it [00:02,  1.19s/it]

...

IoU: 0.9449904656383231
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_bed_train_noneighbor/output/55999/test/mesh/7a4e07fb5701172c862eec8232fff1e_scene0560_00_ins_2.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/02818832/7a4e07fb5701172c862eec8232fff1e_scene0560_00_ins_2/model_canonical_manifoldplus.obj


25it [00:29,  1.10s/it]

IoU: 0.6757027568542118
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_bed_train_noneighbor/output/55999/test/mesh/76db17c76f828282dcb2f14e2e42ec8d_scene0656_03_ins_1.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/02818832/76db17c76f828282dcb2f14e2e42ec8d_scene0656_03_ins_1/model_canonical_manifoldplus.obj


26it [00:31,  1.20s/it]

IoU: 0.7034629879537297


In [7]:
try:
    del iou_path
    del mesh_list_path
except:
    print("no previous paths!")

ious = np.array(ious)
print(ious.mean())
iou_path = os.path.join(os.path.dirname(output_root),"ious.txt")
np.savetxt(iou_path, ious)

mesh_list_path = os.path.join(os.path.dirname(output_root),"mesh_list.txt")
with open(mesh_list_path,"w") as f:
    for imesh in mesh_list:
        f.write(imesh+"\n")

no previous paths!
0.6803794009105846


## ddit_stage2_bed

In [10]:
label = "bed"
gt_root = "/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus"
gt_root = os.path.join(gt_root,label2id[label])

sdf_root = "/storage/user/huju/transferred/ws_dditnach/DeepSDF/data/SdfSamples/canonical_mesh_manifoldplus" # 加载相关的transformation

# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/stage2_diff_cond_scanarcw/recon/49999/Meshes"
# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond/recon/23999/Meshes"
# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_neighbor/output/49999/test/mesh/Meshes"
# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_noneighbor/output/69999/test/mesh/Meshes"
# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/13999/test/mesh/Meshes"
# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/9999/test/mesh/Meshes"
# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/dimr_stage2_sofa/output/1199/test/mesh/Meshes"
# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_chair_train_noneighbor/output/18999/test/mesh/oldMeshes"
# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_chair/output/999/test/mesh/Meshes"

# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_bed_train_noneighbor/output/55999/test/mesh"

output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_bed/output/999/test/mesh"

mesh_list = os.listdir(output_root)
ious = []
base_names = []

for id_mesh, mesh in tqdm(enumerate(mesh_list)):

    mesh_path1 = os.path.join(output_root,mesh)

    base_name = os.path.basename(mesh_path1).split(".")[0]
    base_names.append(base_name)

    mesh_id, scene1, scene2, _,  obj_id = base_name.split(".")[0].split("_")
    
    # pdb.set_trace()
    # 加载两个mesh文件
    # mesh1 = load_mesh(mesh_path1)
    mesh1 = load_mesh(mesh_path1)

    # mesh_path2 = os.path.join(gt_root,base_name, "model_normalized_manifoldplus.obj")
    mesh_path2 = os.path.join(gt_root,base_name,"model_canonical_manifoldplus.obj")
    mesh2 = load_mesh(mesh_path2)
    
    sdf_path = os.path.join(sdf_root,label2id[label],base_name+".npz")
    sdf = np.load(sdf_path)
    translation_mesh2sdf = sdf['translation_mesh2sdf']
    scale_mesh2sdf = sdf["scale_mesh2sdf"]

    mesh1 = mesh_apply_rts(mesh1,mesh_name="recon",mesh_color="rgba(22,244,244,0.5)")
    # mesh2 = mesh_apply_rts(mesh2,translation_c2w=translation_mesh2sdf,scale_c2w=scale_mesh2sdf,mesh_name="gt")
    mesh2, scale, translation = BoundingCubeNormalization(mesh2)
    # mesh2 = mesh_apply_rts(mesh2,mesh_name="gt")

    # input()

    print(mesh_path1)
    print(mesh_path2)

    # vis_data = [mesh1,mesh2]

    # layout = go.Layout(scene=dict(
    #         aspectmode='data',  # Set the aspect ratio to 'cube' for equal scales
    #         xaxis_title='X',
    #         yaxis_title='Y',
    #         zaxis_title='Z',
    #     ),)

    # fig = go.Figure()
    # fig = go.Figure(data=vis_data, layout=layout)
    # fig.show()

    # 计算IoU
    mesh1 = plotly_mesh3d_to_trimesh(mesh1)
    mesh2 = plotly_mesh3d_to_trimesh(mesh2)
    iou = compute_iou(mesh1, mesh2)
    print(f"IoU: {iou}")
    ious.append(iou)

    # input()

0it [00:00, ?it/s]

/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_bed/output/999/test/mesh/946bd5aeda453b38b3a454ed6a7199e2_scene0426_03_ins_2.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/02818832/946bd5aeda453b38b3a454ed6a7199e2_scene0426_03_ins_2/model_canonical_manifoldplus.obj


1it [00:04,  4.10s/it]

IoU: 0.7443848375596368
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_bed/output/999/test/mesh/51223b1b770ff5e72f38f5bd71072746_scene0376_01_ins_4.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/02818832/51223b1b770ff5e72f38f5bd71072746_scene0376_01_ins_4/model_canonical_manifoldplus.obj


2it [00:08,  4.49s/it]

...

IoU: 0.7894274565759117
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_bed/output/999/test/mesh/7a4e07fb5701172c862eec8232fff1e_scene0560_00_ins_2.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/02818832/7a4e07fb5701172c862eec8232fff1e_scene0560_00_ins_2/model_canonical_manifoldplus.obj


25it [00:57,  1.96s/it]

IoU: 0.850123222966342
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_bed/output/999/test/mesh/76db17c76f828282dcb2f14e2e42ec8d_scene0656_03_ins_1.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/02818832/76db17c76f828282dcb2f14e2e42ec8d_scene0656_03_ins_1/model_canonical_manifoldplus.obj


26it [00:59,  2.30s/it]

IoU: 0.7833120855690808


In [11]:
try:
    del iou_path
    del mesh_list_path
except:
    print("no previous paths!")

ious = np.array(ious)
print(ious.mean())
iou_path = os.path.join(os.path.dirname(output_root),"ious.txt")
print(iou_path)
assert(not os.path.exists(iou_path))
np.savetxt(iou_path, ious)

mesh_list_path = os.path.join(os.path.dirname(output_root),"mesh_list.txt")
with open(mesh_list_path,"w") as f:
    for imesh in mesh_list:
        f.write(imesh+"\n")

0.7827004388374376
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_bed/output/999/test/ious.txt


## dimr_stage2_bed

In [12]:
label = "bed"
gt_root = "/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus"
gt_root = os.path.join(gt_root,label2id[label])

sdf_root = "/storage/user/huju/transferred/ws_dditnach/DeepSDF/data/SdfSamples/canonical_mesh_manifoldplus" # 加载相关的transformation

# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/stage2_diff_cond_scanarcw/recon/49999/Meshes"
# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond/recon/23999/Meshes"
# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_neighbor/output/49999/test/mesh/Meshes"
# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_noneighbor/output/69999/test/mesh/Meshes"
# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/13999/test/mesh/Meshes"
# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/9999/test/mesh/Meshes"
# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/dimr_stage2_sofa/output/1199/test/mesh/Meshes"
# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_chair_train_noneighbor/output/18999/test/mesh/oldMeshes"
# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_chair/output/999/test/mesh/Meshes"

# output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_bed_train_noneighbor/output/55999/test/mesh"

out = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_bed/output/999/test/mesh"
output_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/dimr_stage2_bed/output/999/test/mesh"

mesh_list = os.listdir(output_root)
ious = []
base_names = []

for id_mesh, mesh in tqdm(enumerate(mesh_list)):

    mesh_path1 = os.path.join(output_root,mesh)

    base_name = os.path.basename(mesh_path1).split(".")[0]
    base_names.append(base_name)

    mesh_id, scene1, scene2, _,  obj_id = base_name.split(".")[0].split("_")
    
    # pdb.set_trace()
    # 加载两个mesh文件
    # mesh1 = load_mesh(mesh_path1)
    mesh1 = load_mesh(mesh_path1)

    # mesh_path2 = os.path.join(gt_root,base_name, "model_normalized_manifoldplus.obj")
    mesh_path2 = os.path.join(gt_root,base_name,"model_canonical_manifoldplus.obj")
    mesh2 = load_mesh(mesh_path2)
    
    sdf_path = os.path.join(sdf_root,label2id[label],base_name+".npz")
    sdf = np.load(sdf_path)
    translation_mesh2sdf = sdf['translation_mesh2sdf']
    scale_mesh2sdf = sdf["scale_mesh2sdf"]

    mesh1 = mesh_apply_rts(mesh1,mesh_name="recon",mesh_color="rgba(22,244,244,0.5)")
    # mesh2 = mesh_apply_rts(mesh2,translation_c2w=translation_mesh2sdf,scale_c2w=scale_mesh2sdf,mesh_name="gt")
    mesh2, scale, translation = BoundingCubeNormalization(mesh2)
    # mesh2 = mesh_apply_rts(mesh2,mesh_name="gt")

    # input()

    print(mesh_path1)
    print(mesh_path2)

    # vis_data = [mesh1,mesh2]

    # layout = go.Layout(scene=dict(
    #         aspectmode='data',  # Set the aspect ratio to 'cube' for equal scales
    #         xaxis_title='X',
    #         yaxis_title='Y',
    #         zaxis_title='Z',
    #     ),)

    # fig = go.Figure()
    # fig = go.Figure(data=vis_data, layout=layout)
    # fig.show()

    # 计算IoU
    mesh1 = plotly_mesh3d_to_trimesh(mesh1)
    mesh2 = plotly_mesh3d_to_trimesh(mesh2)
    iou = compute_iou(mesh1, mesh2)
    print(f"IoU: {iou}")
    ious.append(iou)

    # input()

0it [00:00, ?it/s]

/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/dimr_stage2_bed/output/999/test/mesh/946bd5aeda453b38b3a454ed6a7199e2_scene0426_03_ins_2.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/02818832/946bd5aeda453b38b3a454ed6a7199e2_scene0426_03_ins_2/model_canonical_manifoldplus.obj


1it [00:00,  1.24it/s]

IoU: 0.4066339467161209
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/dimr_stage2_bed/output/999/test/mesh/51223b1b770ff5e72f38f5bd71072746_scene0376_01_ins_4.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/02818832/51223b1b770ff5e72f38f5bd71072746_scene0376_01_ins_4/model_canonical_manifoldplus.obj


2it [00:01,  1.26it/s]

...

IoU: 0.7256588392991283
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/dimr_stage2_bed/output/999/test/mesh/7a4e07fb5701172c862eec8232fff1e_scene0560_00_ins_2.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/02818832/7a4e07fb5701172c862eec8232fff1e_scene0560_00_ins_2/model_canonical_manifoldplus.obj


25it [00:50,  1.01s/it]

IoU: 0.8330160609813971
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/dimr_stage2_bed/output/999/test/mesh/76db17c76f828282dcb2f14e2e42ec8d_scene0656_03_ins_1.ply.ply
/storage/user/huju/transferred/ws_dditnach/DATA/ScanARCW/canonical_mesh_manifoldplus/02818832/76db17c76f828282dcb2f14e2e42ec8d_scene0656_03_ins_1/model_canonical_manifoldplus.obj


26it [00:51,  1.99s/it]

IoU: 0.6307033211684122


In [13]:
try:
    del iou_path
    del mesh_list_path
except:
    print("no previous paths!")

ious = np.array(ious)
print(ious.mean())
iou_path = os.path.join(os.path.dirname(output_root),"ious.txt")
print(iou_path)
assert(not os.path.exists(iou_path))
np.savetxt(iou_path, ious)

mesh_list_path = os.path.join(os.path.dirname(output_root),"mesh_list.txt")
with open(mesh_list_path,"w") as f:
    for imesh in mesh_list:
        f.write(imesh+"\n")

0.6467237289753881
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/dimr_stage2_bed/output/999/test/ious.txt
